In [4]:
!pip install azure-storage-blob
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.4/185.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.7 MB/s eta 0:00:00


In [5]:
!pip install feedparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=4b8da68c076c6e31a20f91a2c8a28a55ea36aa452164c67a7313ae33dacd9dfc
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [6]:
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=f3aa0e191435d3c4070238d01d6a38c13dccfee46ed2e0208fb681b3042243d7
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3339 sha256=6be1e366b9051b40fb8e618f73028d008320acf01cbac09dd54de61ea14ddc86
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc933e41c4ccd6a8f8edb
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398380 sha256=8e42c01ae322

In [1]:
!pip install spacy

In [2]:
!python -m spacy download en_core_web_sm

2023-07-31 00:46:04.187980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 00:46:05.492254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [13]:
import feedparser
from newspaper import Article
from textblob import TextBlob
import pandas as pd
import spacy
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Load spaCy language model
nlp = spacy.load('en_core_web_sm')

In [8]:
# Function to extract articles from an RSS feed and perform sentiment analysis
def extract_articles_from_rss(feed_url):
    try:
        feed = feedparser.parse(feed_url)
        articles = []
        for entry in feed.entries:
            url = entry.link
            article = Article(url)
            article.download()
            article.parse()

            title = article.title.strip()
            link = url.strip()
            content = article.text.strip()

            blob = TextBlob(content)
            sentiment_score = blob.sentiment.polarity

            # Extract image link (if present)
            image_link = article.top_image if hasattr(article, 'top_image') else None

            # Extract publication year
            publication_year = None
            if 'published_parsed' in entry:
                publication_year = entry.published_parsed.tm_year

             # Perform Keyword Extraction using TextBlob
            keyword_blob = TextBlob(content)
            keywords = keyword_blob.noun_phrases

            # Perform Entity Recognition using spaCy
            doc = nlp(content)
            entities = [(ent.text, ent.label_) for ent in doc.ents]

            articles.append({
                'Title': title,
                'Link': link,
                'Content': content,
                'Sentiment Score': sentiment_score,
                'Image Link': image_link,
                'Publication Year': publication_year,
                'Keywords': keywords,
                'Entities': entities
            })

            # Print the article details
            print(f"Title: {title}")
            print(f"Link: {link}")

        return articles
    except Exception as e:
        print(f"An error occurred while processing {feed_url}: {e}")
        return []

In [11]:
# Example list of RSS feeds for specific sections (Politics, Science, Business) from real websites
rss_feeds = {
    'https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml': 'Politics',
    'http://rss.cnn.com/rss/cnn_allpolitics.rss': 'Politics',

    'https://rss.nytimes.com/services/xml/rss/nyt/Science.xml': 'Science',
    'http://rss.cnn.com/rss/edition_space.rss': 'Science',

    'https://rss.nytimes.com/services/xml/rss/nyt/Business.xml': 'Business',
    'http://rss.cnn.com/rss/money_news_international.rss': 'Business'
}

# Extract articles from RSS feeds
all_articles = []
for feed_url, section in rss_feeds.items():
    print(f"Fetching articles from {section} section:")
    articles = extract_articles_from_rss(feed_url)
    for article in articles:
        article['Section'] = section
    all_articles.extend(articles)

Fetching articles from Politics section:
Title: The Secret History of Gun Rights: How Lawmakers Armed the N.R.A.
Link: https://www.nytimes.com/2023/07/30/us/politics/nra-congress-firearms.html
Title: The Legacy Dilemma: What to Do About Privileges for the Privileged?
Link: https://www.nytimes.com/2023/07/30/us/politics/legacy-admissions-college-alumni.html
Title: A Left-Right Alliance Puts Iowa’s CO2 Pipelines on the Presidential Agenda
Link: https://www.nytimes.com/2023/07/30/us/politics/iowa-pipelines-trump.html
Title: Nuts-and-Bolts Conservatism From Nikki Haley
Link: https://www.nytimes.com/2023/07/30/us/politics/nikki-haley-stump-speech.html
Title: Trump Team Creates Legal-Defense Fund to Cover His Allies’ Bills
Link: https://www.nytimes.com/2023/07/30/us/politics/trump-legal-defense-fund.html
Title: Minor Characters Emerge to Play Key Roles in Trump Documents Case
Link: https://www.nytimes.com/2023/07/30/us/politics/trump-documents-de-oliveira-nauta.html
Title: Judge Dismisses Tr

In [14]:
# Create a pandas DataFrame from the extracted data
df = pd.DataFrame(all_articles)

def send_to_azure_blob_storage(df, container_name, blob_name, connection_string):
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        container_client = blob_service_client.get_container_client(container_name)

        blob_data = df.to_csv(index=False)
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(blob_data)

        print(f"Uploaded to Azure Blob Storage in container '{container_name}' as '{blob_name}'.")
    except Exception as e:
        print(f"An error occurred during upload to Azure Blob Storage: {e}")


container_name = 'assn4'
blob_name = 'scrapedData.csv'
connection_string = 'DefaultEndpointsProtocol=https;AccountName=assn4strg;AccountKey=KSeVmhAXiNTMvX5ZFuSWRA/B4BW2erjJ+8d59hkMmmH7LwTX6RTWczx3Yf0vaUr9iGuRprgjZx7F+AStvI4h5Q==;EndpointSuffix=core.windows.net'

# Send the DataFrame as CSV to Azure Blob Storage
send_to_azure_blob_storage(df, container_name, blob_name, connection_string)

Uploaded to Azure Blob Storage in container 'assn4' as 'scrapedData.csv'.
